In [1]:
# Import our dependencies

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Import dataset

charity_df = pd.read_csv('charity_data.csv')
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Find null values

for column in charity_df.columns: 
    print(f"Column {column} has {charity_df[column].isnull().sum()} null values")

Column EIN has 0 null values
Column NAME has 0 null values
Column APPLICATION_TYPE has 0 null values
Column AFFILIATION has 0 null values
Column CLASSIFICATION has 0 null values
Column USE_CASE has 0 null values
Column ORGANIZATION has 0 null values
Column STATUS has 0 null values
Column INCOME_AMT has 0 null values
Column SPECIAL_CONSIDERATIONS has 0 null values
Column ASK_AMT has 0 null values
Column IS_SUCCESSFUL has 0 null values


In [4]:
# Find duplicate entries
print(f"Duplicate entries: {charity_df.duplicated().sum()}")

Duplicate entries: 0


In [5]:
# Generate categorical variable list

charity_category = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
charity_df[charity_category].nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
dtype: int64

In [6]:
# Print out the EIN value counts
EIN_counts = charity_df.EIN.value_counts()
EIN_counts

471361061    1
716145914    1
470850777    1
742622426    1
436124893    1
            ..
205040320    1
461149136    1
474460867    1
204833477    1
352260098    1
Name: EIN, Length: 34299, dtype: int64

In [7]:
# Remove EIN - this column does not have any berring on the analysis
charity_df.drop(columns=["EIN"], inplace=True)
charity_df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [8]:
# Print out the NAME value counts
name_counts = charity_df.NAME.value_counts()
name_counts.head(30)

PARENT BOOSTER USA INC                                                1260
TOPS CLUB INC                                                          765
UNITED STATES BOWLING CONGRESS INC                                     700
WASHINGTON STATE UNIVERSITY                                            492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                        408
PTA TEXAS CONGRESS                                                     368
SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC                          331
ALPHA PHI SIGMA                                                        313
TOASTMASTERS INTERNATIONAL                                             293
MOST WORSHIPFUL STRINGER FREE AND ACCEPTED MASONS                      287
LITTLE LEAGUE BASEBALL INC                                             277
INTERNATIONAL ASSOCIATION OF LIONS CLUBS                               266
MOMS CLUB                                                              210
INTERNATIONAL ASSOCIATION

In [9]:
# Determine which name values to replace
replace_name = list(name_counts[name_counts < 200].index)

# Replace in DataFrame
for NAME in replace_name:
    charity_df.NAME = charity_df.NAME.replace(NAME,"Other")


# Check to make sure binning was successful
charity_df.NAME.value_counts()

Other                                                                 28123
PARENT BOOSTER USA INC                                                 1260
TOPS CLUB INC                                                           765
UNITED STATES BOWLING CONGRESS INC                                      700
WASHINGTON STATE UNIVERSITY                                             492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                         408
PTA TEXAS CONGRESS                                                      368
SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC                           331
ALPHA PHI SIGMA                                                         313
TOASTMASTERS INTERNATIONAL                                              293
MOST WORSHIPFUL STRINGER FREE AND ACCEPTED MASONS                       287
LITTLE LEAGUE BASEBALL INC                                              277
INTERNATIONAL ASSOCIATION OF LIONS CLUBS                                266
MOMS CLUB   

In [10]:
# Print out the Classification value counts
classification_counts = charity_df.CLASSIFICATION.value_counts()
classification_counts.head(30)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1278       10
C1238       10
C1237        9
Name: CLASSIFICATION, dtype: int64

In [11]:
# Determine which classification values to replace
replace_classification = list(classification_counts[classification_counts < 700].index)

# Replace in DataFrame
for CLASSIFICATION in replace_classification:
    charity_df.CLASSIFICATION = charity_df.CLASSIFICATION.replace(CLASSIFICATION,"Other")


# Check to make sure binning was successful
charity_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [12]:
# Print out the application type value counts
applicationtype_counts = charity_df.APPLICATION_TYPE.value_counts()
applicationtype_counts.head(30)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [13]:

# Determine which application type values to replace
replace_applicationtype = list(applicationtype_counts[applicationtype_counts < 100].index)

# Replace in DataFrame
for APPLICATION_TYPE in replace_applicationtype:
    charity_df.APPLICATION_TYPE = charity_df.APPLICATION_TYPE.replace(APPLICATION_TYPE,"Other")


# Check to make sure binning was successful
charity_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [14]:
# Print out the income amount type value counts
incomeamount_counts = charity_df.INCOME_AMT.value_counts()
incomeamount_counts.head(30)

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [15]:
# Create the OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit the encoder and produce encoded DataFrame
encode_df = pd.DataFrame(enc.fit_transform(charity_df.APPLICATION_TYPE.values.reshape(-1,1)))

# Rename encoded columns
encode_df.columns = enc.get_feature_names(['APPLICATION_TYPE'])
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,APPLICATION_TYPE_T9
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# Merge the two DataFrames together and drop the APPLICATION_TYPE column
df1 = charity_df.merge(encode_df,left_index=True,right_index=True).drop("APPLICATION_TYPE",1)
df1

,NAME,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,APPLICATION_TYPE_T9
0,Other,Independent,C1000,ProductDev,Association,1,0,N,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Other,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Other,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,Other,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Other,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,Other,Independent,C1000,ProductDev,Association,1,0,N,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
34295,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
34296,Other,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
34297,Other,Independent,C3000,ProductDev,Association,1,0,N,5000,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [17]:
# Print out the AFFILIATION value counts
AFFILIATION_counts = charity_df.AFFILIATION.value_counts()
AFFILIATION_counts

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [18]:

# Create the OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit the encoder and produce encoded DataFrame
encode_df = pd.DataFrame(enc.fit_transform(charity_df.AFFILIATION.values.reshape(-1,1)))

# Rename encoded columns
encode_df.columns = enc.get_feature_names(['AFFILIATION'])
encode_df.head()

,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional
0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0


In [19]:
# Merge the two DataFrames together and drop the AFFILIATION column
df2 = df1.merge(encode_df,left_index=True,right_index=True).drop("AFFILIATION",1)
df2

,NAME,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,...,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,APPLICATION_TYPE_T9,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional
0,Other,C1000,ProductDev,Association,1,0,N,5000,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,Other,C2000,Preservation,Co-operative,1,1-9999,N,108590,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,Other,C3000,ProductDev,Association,1,0,N,5000,0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,Other,C2000,Preservation,Trust,1,10000-24999,N,6692,1,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,Other,C1000,Heathcare,Trust,1,100000-499999,N,142590,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,Other,C1000,ProductDev,Association,1,0,N,5000,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
34295,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,C3000,ProductDev,Association,1,0,N,5000,0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
34296,Other,C2000,Preservation,Association,1,0,N,5000,0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
34297,Other,C3000,ProductDev,Association,1,0,N,5000,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [20]:
# Create the OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit the encoder and produce encoded DataFrame
encode_df = pd.DataFrame(enc.fit_transform(df2.CLASSIFICATION.values.reshape(-1,1)))

# Rename encoded columns
encode_df.columns = enc.get_feature_names(['CLASSIFICATION'])
encode_df.head()

,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_C7000,CLASSIFICATION_Other
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
# Merge the two DataFrames together and drop the CLASSIFICATION column
df3 = df2.merge(encode_df,left_index=True,right_index=True).drop("CLASSIFICATION",1)
df3

,NAME,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,...,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_C7000,CLASSIFICATION_Other
0,Other,ProductDev,Association,1,0,N,5000,1,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Other,Preservation,Co-operative,1,1-9999,N,108590,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Other,ProductDev,Association,1,0,N,5000,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,Other,Preservation,Trust,1,10000-24999,N,6692,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,Other,Heathcare,Trust,1,100000-499999,N,142590,1,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,Other,ProductDev,Association,1,0,N,5000,0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
34295,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,ProductDev,Association,1,0,N,5000,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
34296,Other,Preservation,Association,1,0,N,5000,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
34297,Other,ProductDev,Association,1,0,N,5000,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [22]:
# Print out the USE_CASE value counts
USE_CASE_counts = charity_df.USE_CASE.value_counts()
USE_CASE_counts

Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64

In [23]:
# Create the OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit the encoder and produce encoded DataFrame
encode_df = pd.DataFrame(enc.fit_transform(df3.USE_CASE.values.reshape(-1,1)))

# Rename encoded columns
encode_df.columns = enc.get_feature_names(['USE_CASE'])
encode_df.head()

,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev
0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0


In [24]:
# Merge the two DataFrames together and drop the USE_CASE column
df4 = df3.merge(encode_df,left_index=True,right_index=True).drop("USE_CASE",1)
df4

,NAME,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,...,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_C7000,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev
0,Other,Association,1,0,N,5000,1,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Other,Co-operative,1,1-9999,N,108590,1,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,Other,Association,1,0,N,5000,0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,Other,Trust,1,10000-24999,N,6692,1,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,Other,Trust,1,100000-499999,N,142590,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,Other,Association,1,0,N,5000,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
34295,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,Association,1,0,N,5000,0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
34296,Other,Association,1,0,N,5000,0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34297,Other,Association,1,0,N,5000,1,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [25]:
# Print out the ORGANIZATION value counts
ORGANIZATION_counts = charity_df.ORGANIZATION.value_counts()
ORGANIZATION_counts

Trust           23515
Association     10255
Co-operative      486
Corporation        43
Name: ORGANIZATION, dtype: int64

In [26]:
# Create the OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit the encoder and produce encoded DataFrame
encode_df = pd.DataFrame(enc.fit_transform(df4.ORGANIZATION.values.reshape(-1,1)))

# Rename encoded columns
encode_df.columns = enc.get_feature_names(['ORGANIZATION'])
encode_df.head()

,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust
0,1.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0


In [27]:

# Merge the two DataFrames together and drop the ORGANIZATION column
df5 = df4.merge(encode_df,left_index=True,right_index=True).drop("ORGANIZATION",1)
df5

,NAME,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,...,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust
0,Other,1,0,N,5000,1,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,Other,1,1-9999,N,108590,1,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,Other,1,0,N,5000,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,Other,1,10000-24999,N,6692,1,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,Other,1,100000-499999,N,142590,1,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,Other,1,0,N,5000,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
34295,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,1,0,N,5000,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
34296,Other,1,0,N,5000,0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
34297,Other,1,0,N,5000,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [28]:
# Print out the STATUS value counts
STATUS_counts = charity_df.STATUS.value_counts()
STATUS_counts

1    34294
0        5
Name: STATUS, dtype: int64

In [29]:
# Create the OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit the encoder and produce encoded DataFrame
encode_df = pd.DataFrame(enc.fit_transform(df5.STATUS.values.reshape(-1,1)))

# Rename encoded columns
encode_df.columns = enc.get_feature_names(['STATUS'])
encode_df.head()

,STATUS_0,STATUS_1
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0


In [30]:
# Merge the two DataFrames together and drop the STATUS column
df6 = df5.merge(encode_df,left_index=True,right_index=True).drop("STATUS",1)
df6

,NAME,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,...,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,STATUS_0,STATUS_1
0,Other,0,N,5000,1,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
1,Other,1-9999,N,108590,1,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,Other,0,N,5000,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
3,Other,10000-24999,N,6692,1,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,Other,100000-499999,N,142590,1,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,Other,0,N,5000,0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
34295,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,0,N,5000,0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
34296,Other,0,N,5000,0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
34297,Other,0,N,5000,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0


In [31]:
# Create the OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit the encoder and produce encoded DataFrame
encode_df = pd.DataFrame(enc.fit_transform(df6.INCOME_AMT.values.reshape(-1,1)))

# Rename encoded columns
encode_df.columns = enc.get_feature_names(['INCOME_AMT'])
encode_df.head()

,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [32]:
# Merge the two DataFrames together and drop the INCOME_AMT column
df7 = df6.merge(encode_df,left_index=True,right_index=True).drop("INCOME_AMT",1)
df7

,NAME,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,...,STATUS_1,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,Other,N,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Other,N,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Other,N,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Other,N,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Other,N,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,Other,N,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34295,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,N,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34296,Other,N,5000,0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34297,Other,N,5000,1,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
# Print out the SPECIAL_CONSIDERATIONS value counts
SPECIAL_CONSIDERATIONS_counts = charity_df.SPECIAL_CONSIDERATIONS.value_counts()
SPECIAL_CONSIDERATIONS_counts

N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [34]:
# Create the OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit the encoder and produce encoded DataFrame
encode_df = pd.DataFrame(enc.fit_transform(df7.SPECIAL_CONSIDERATIONS.values.reshape(-1,1)))

# Rename encoded columns
encode_df.columns = enc.get_feature_names(['SPECIAL_CONSIDERATIONS'])
encode_df.head()

,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0


In [35]:

# Merge the two DataFrames together and drop the SPECIAL_CONSIDERATIONS column
df8 = df7.merge(encode_df,left_index=True,right_index=True).drop("SPECIAL_CONSIDERATIONS",1)
df8

,NAME,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,Other,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,Other,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,Other,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,Other,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,Other,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,Other,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34295,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34296,Other,5000,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34297,Other,5000,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [36]:
# Print out the ASK_AMT value counts
ASK_AMT_counts = charity_df.ASK_AMT.value_counts()
ASK_AMT_counts

5000       25398
10478          3
15583          3
6725           3
63981          3
           ...  
772556         1
70103          1
27096          1
25049          1
1138700        1
Name: ASK_AMT, Length: 8747, dtype: int64

In [37]:
# Remove the column
df8.drop(columns=["ASK_AMT"], inplace=True)
df8.head()

,NAME,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,Other,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,Other,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,Other,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,Other,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,Other,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [38]:

# Print out the IS_SUCCESSFUL value counts
IS_SUCCESSFUL_counts = charity_df.IS_SUCCESSFUL.value_counts()
IS_SUCCESSFUL_counts

1    18261
0    16038
Name: IS_SUCCESSFUL, dtype: int64

In [39]:
# Create the OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit the encoder and produce encoded DataFrame
encode_df = pd.DataFrame(enc.fit_transform(df8.IS_SUCCESSFUL.values.reshape(-1,1)))

# Rename encoded columns
encode_df.columns = enc.get_feature_names(['IS_SUCCESSFUL'])
encode_df.head()

,IS_SUCCESSFUL_0,IS_SUCCESSFUL_1
0,0.0,1.0
1,0.0,1.0
2,1.0,0.0
3,0.0,1.0
4,0.0,1.0


In [40]:
# Merge the two DataFrames together and drop the IS_SUCCESSFUL column
df = df8.merge(encode_df,left_index=True,right_index=True).drop("IS_SUCCESSFUL",1)
df

,NAME,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,IS_SUCCESSFUL_0,IS_SUCCESSFUL_1
0,Other,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,Other,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,Other,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,Other,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,Other,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,Other,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
34295,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
34296,Other,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
34297,Other,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [41]:
# Drop NAME column

clean_df = df.drop(columns=["NAME"])
clean_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,APPLICATION_TYPE_T9,...,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,IS_SUCCESSFUL_0,IS_SUCCESSFUL_1
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [42]:
# Generate our categorical variable list
df_cat = clean_df.dtypes[df.dtypes == "float64"].index.tolist()
df_cat

['APPLICATION_TYPE_Other',
 'APPLICATION_TYPE_T10',
 'APPLICATION_TYPE_T19',
 'APPLICATION_TYPE_T3',
 'APPLICATION_TYPE_T4',
 'APPLICATION_TYPE_T5',
 'APPLICATION_TYPE_T6',
 'APPLICATION_TYPE_T7',
 'APPLICATION_TYPE_T8',
 'APPLICATION_TYPE_T9',
 'AFFILIATION_CompanySponsored',
 'AFFILIATION_Family/Parent',
 'AFFILIATION_Independent',
 'AFFILIATION_National',
 'AFFILIATION_Other',
 'AFFILIATION_Regional',
 'CLASSIFICATION_C1000',
 'CLASSIFICATION_C1200',
 'CLASSIFICATION_C2000',
 'CLASSIFICATION_C2100',
 'CLASSIFICATION_C3000',
 'CLASSIFICATION_C7000',
 'CLASSIFICATION_Other',
 'USE_CASE_CommunityServ',
 'USE_CASE_Heathcare',
 'USE_CASE_Other',
 'USE_CASE_Preservation',
 'USE_CASE_ProductDev',
 'ORGANIZATION_Association',
 'ORGANIZATION_Co-operative',
 'ORGANIZATION_Corporation',
 'ORGANIZATION_Trust',
 'STATUS_0',
 'STATUS_1',
 'INCOME_AMT_0',
 'INCOME_AMT_1-9999',
 'INCOME_AMT_10000-24999',
 'INCOME_AMT_100000-499999',
 'INCOME_AMT_10M-50M',
 'INCOME_AMT_1M-5M',
 'INCOME_AMT_25000-999

In [43]:
# Check the number of unique values in each column
df[df_cat].nunique()

APPLICATION_TYPE_Other          2
APPLICATION_TYPE_T10            2
APPLICATION_TYPE_T19            2
APPLICATION_TYPE_T3             2
APPLICATION_TYPE_T4             2
APPLICATION_TYPE_T5             2
APPLICATION_TYPE_T6             2
APPLICATION_TYPE_T7             2
APPLICATION_TYPE_T8             2
APPLICATION_TYPE_T9             2
AFFILIATION_CompanySponsored    2
AFFILIATION_Family/Parent       2
AFFILIATION_Independent         2
AFFILIATION_National            2
AFFILIATION_Other               2
AFFILIATION_Regional            2
CLASSIFICATION_C1000            2
CLASSIFICATION_C1200            2
CLASSIFICATION_C2000            2
CLASSIFICATION_C2100            2
CLASSIFICATION_C3000            2
CLASSIFICATION_C7000            2
CLASSIFICATION_Other            2
USE_CASE_CommunityServ          2
USE_CASE_Heathcare              2
USE_CASE_Other                  2
USE_CASE_Preservation           2
USE_CASE_ProductDev             2
ORGANIZATION_Association        2
ORGANIZATION_C

In [49]:
# Split our preprocessed data into our features and target arrays
y = clean_df["IS_SUCCESSFUL_0"].values
X = clean_df.drop(["IS_SUCCESSFUL_0","IS_SUCCESSFUL_1"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)


In [50]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [51]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 368       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 419
Trainable params: 419
Non-trainable params: 0
_________________________________________________________________


In [52]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [53]:

# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 2s 88us/sample - loss: 0.6128 - accuracy: 0.6864
Epoch 2/100
25724/25724 [==============================] - 1s 53us/sample - loss: 0.5699 - accuracy: 0.7250
Epoch 3/100
25724/25724 [==============================] - 1s 51us/sample - loss: 0.5607 - accuracy: 0.7303
Epoch 4/100
25724/25724 [==============================] - 1s 47us/sample - loss: 0.5574 - accuracy: 0.7309
Epoch 5/100
25724/25724 [==============================] - 1s 45us/sample - loss: 0.5558 - accuracy: 0.7306
Epoch 6/100
25724/25724 [==============================] - 1s 45us/sample - loss: 0.5544 - accuracy: 0.7306
Epoch 7/100
25724/25724 [==============================] - 1s 44us/sample - loss: 0.5532 - accuracy: 0.7308
Epoch 8/100
25724/25724 [==============================] - 1s 45us/sample - loss: 0.5523 - accuracy: 0.7311
Epoch 9/100
25724/25724 [==============================] - 1s 44us/sample - loss: 0.5513 - accuracy: 0.7323
Epoch

25724/25724 [==============================] - 1s 51us/sample - loss: 0.5439 - accuracy: 0.7348
Epoch 76/100
25724/25724 [==============================] - 1s 51us/sample - loss: 0.5442 - accuracy: 0.7346
Epoch 77/100
25724/25724 [==============================] - 1s 51us/sample - loss: 0.5441 - accuracy: 0.7342
Epoch 78/100
25724/25724 [==============================] - 1s 51us/sample - loss: 0.5439 - accuracy: 0.7353
Epoch 79/100
25724/25724 [==============================] - 1s 52us/sample - loss: 0.5436 - accuracy: 0.7345
Epoch 80/100
25724/25724 [==============================] - 1s 51us/sample - loss: 0.5439 - accuracy: 0.7352
Epoch 81/100
25724/25724 [==============================] - 1s 52us/sample - loss: 0.5440 - accuracy: 0.7358
Epoch 82/100
25724/25724 [==============================] - 1s 52us/sample - loss: 0.5438 - accuracy: 0.7345
Epoch 83/100
25724/25724 [==============================] - 1s 54us/sample - loss: 0.5435 - accuracy: 0.7348
Epoch 84/100
25724/25724 [======

In [54]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/1 - 0s - loss: 0.5517 - accuracy: 0.7257
Loss: 0.5511228469439915, Accuracy: 0.7257142663002014


In [55]:
# Define model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 368       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 419
Trainable params: 419
Non-trainable params: 0
_________________________________________________________________


# Optimization - Adding neurons

In [69]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  12
hidden_nodes_layer2 = 6

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 12)                552       
_________________________________________________________________
dense_22 (Dense)             (None, 6)                 78        
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 7         
Total params: 637
Trainable params: 637
Non-trainable params: 0
_________________________________________________________________


In [70]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [71]:

# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 2s 86us/sample - loss: 0.5977 - accuracy: 0.7044
Epoch 2/100
25724/25724 [==============================] - 1s 52us/sample - loss: 0.5606 - accuracy: 0.7273
Epoch 3/100
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5556 - accuracy: 0.7289
Epoch 4/100
25724/25724 [==============================] - 1s 47us/sample - loss: 0.5543 - accuracy: 0.7300
Epoch 5/100
25724/25724 [==============================] - 1s 46us/sample - loss: 0.5531 - accuracy: 0.7293
Epoch 6/100
25724/25724 [==============================] - 1s 47us/sample - loss: 0.5521 - accuracy: 0.7310
Epoch 7/100
25724/25724 [==============================] - 1s 46us/sample - loss: 0.5518 - accuracy: 0.7308
Epoch 8/100
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5505 - accuracy: 0.7322
Epoch 9/100
25724/25724 [==============================] - 1s 48us/sample - loss: 0.5503 - accuracy: 0.7316
Epoch

25724/25724 [==============================] - 1s 49us/sample - loss: 0.5425 - accuracy: 0.7368
Epoch 77/100
25724/25724 [==============================] - 1s 47us/sample - loss: 0.5422 - accuracy: 0.7362
Epoch 78/100
25724/25724 [==============================] - 1s 47us/sample - loss: 0.5420 - accuracy: 0.7371
Epoch 79/100
25724/25724 [==============================] - 1s 46us/sample - loss: 0.5421 - accuracy: 0.7360
Epoch 80/100
25724/25724 [==============================] - 1s 47us/sample - loss: 0.5420 - accuracy: 0.7362
Epoch 81/100
25724/25724 [==============================] - 1s 48us/sample - loss: 0.5422 - accuracy: 0.7367
Epoch 82/100
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5419 - accuracy: 0.7360
Epoch 83/100
25724/25724 [==============================] - 1s 48us/sample - loss: 0.5417 - accuracy: 0.7364
Epoch 84/100
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5419 - accuracy: 0.7358
Epoch 85/100
25724/25724 [======

In [72]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/1 - 0s - loss: 0.5504 - accuracy: 0.7293
Loss: 0.5496628469489406, Accuracy: 0.7293294668197632


In [73]:
# Define model
nn.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 12)                552       
_________________________________________________________________
dense_22 (Dense)             (None, 6)                 78        
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 7         
Total params: 637
Trainable params: 637
Non-trainable params: 0
_________________________________________________________________


# Optimization - Adding hidden layer

In [74]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  12
hidden_nodes_layer2 = 6
hidden_nodes_layer3 = 2

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 12)                552       
_________________________________________________________________
dense_25 (Dense)             (None, 6)                 78        
_________________________________________________________________
dense_26 (Dense)             (None, 2)                 14        
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 3         
Total params: 647
Trainable params: 647
Non-trainable params: 0
_________________________________________________________________


In [75]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [76]:

# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 2s 89us/sample - loss: 0.6179 - accuracy: 0.7021
Epoch 2/100
25724/25724 [==============================] - 2s 64us/sample - loss: 0.5796 - accuracy: 0.7301
Epoch 3/100
25724/25724 [==============================] - 2s 59us/sample - loss: 0.5701 - accuracy: 0.7313
Epoch 4/100
25724/25724 [==============================] - 2s 62us/sample - loss: 0.5657 - accuracy: 0.7313
Epoch 5/100
25724/25724 [==============================] - 1s 54us/sample - loss: 0.5619 - accuracy: 0.7325
Epoch 6/100
25724/25724 [==============================] - 1s 57us/sample - loss: 0.5572 - accuracy: 0.7318
Epoch 7/100
25724/25724 [==============================] - 2s 62us/sample - loss: 0.5555 - accuracy: 0.7322
Epoch 8/100
25724/25724 [==============================] - 2s 62us/sample - loss: 0.5544 - accuracy: 0.7315
Epoch 9/100
25724/25724 [==============================] - 2s 64us/sample - loss: 0.5538 - accuracy: 0.7323
Epoch

25724/25724 [==============================] - 2s 63us/sample - loss: 0.5429 - accuracy: 0.7360
Epoch 77/100
25724/25724 [==============================] - 2s 61us/sample - loss: 0.5428 - accuracy: 0.7358
Epoch 78/100
25724/25724 [==============================] - 2s 68us/sample - loss: 0.5430 - accuracy: 0.7365
Epoch 79/100
25724/25724 [==============================] - 2s 60us/sample - loss: 0.5426 - accuracy: 0.7364
Epoch 80/100
25724/25724 [==============================] - 2s 61us/sample - loss: 0.5426 - accuracy: 0.7368
Epoch 81/100
25724/25724 [==============================] - 2s 66us/sample - loss: 0.5425 - accuracy: 0.7372
Epoch 82/100
25724/25724 [==============================] - 2s 60us/sample - loss: 0.5428 - accuracy: 0.7370
Epoch 83/100
25724/25724 [==============================] - 2s 64us/sample - loss: 0.5423 - accuracy: 0.7377
Epoch 84/100
25724/25724 [==============================] - 2s 59us/sample - loss: 0.5423 - accuracy: 0.7366
Epoch 85/100
25724/25724 [======

In [77]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/1 - 1s - loss: 0.5597 - accuracy: 0.7264
Loss: 0.5520887616841508, Accuracy: 0.7264139652252197


In [78]:
# Define model
nn.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 12)                552       
_________________________________________________________________
dense_25 (Dense)             (None, 6)                 78        
_________________________________________________________________
dense_26 (Dense)             (None, 2)                 14        
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 3         
Total params: 647
Trainable params: 647
Non-trainable params: 0
_________________________________________________________________


# Optimization - changing activation function of hidden layers

In [79]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 8)                 368       
_________________________________________________________________
dense_29 (Dense)             (None, 5)                 45        
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 6         
Total params: 419
Trainable params: 419
Non-trainable params: 0
_________________________________________________________________


In [80]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [81]:

# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 2s 86us/sample - loss: 0.6619 - accuracy: 0.6181
Epoch 2/100
25724/25724 [==============================] - 1s 53us/sample - loss: 0.5969 - accuracy: 0.7141
Epoch 3/100
25724/25724 [==============================] - 2s 60us/sample - loss: 0.5802 - accuracy: 0.7228
Epoch 4/100
25724/25724 [==============================] - 2s 59us/sample - loss: 0.5755 - accuracy: 0.7240
Epoch 5/100
25724/25724 [==============================] - 1s 54us/sample - loss: 0.5734 - accuracy: 0.7234
Epoch 6/100
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5719 - accuracy: 0.7247
Epoch 7/100
25724/25724 [==============================] - 2s 59us/sample - loss: 0.5704 - accuracy: 0.7259
Epoch 8/100
25724/25724 [==============================] - 1s 58us/sample - loss: 0.5691 - accuracy: 0.7260
Epoch 9/100
25724/25724 [==============================] - 1s 56us/sample - loss: 0.5677 - accuracy: 0.7270
Epoch

25724/25724 [==============================] - 2s 59us/sample - loss: 0.5488 - accuracy: 0.7330
Epoch 77/100
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5485 - accuracy: 0.7338
Epoch 78/100
25724/25724 [==============================] - 1s 48us/sample - loss: 0.5485 - accuracy: 0.7336
Epoch 79/100
25724/25724 [==============================] - 1s 47us/sample - loss: 0.5483 - accuracy: 0.7341
Epoch 80/100
25724/25724 [==============================] - 1s 47us/sample - loss: 0.5483 - accuracy: 0.7341
Epoch 81/100
25724/25724 [==============================] - 1s 47us/sample - loss: 0.5482 - accuracy: 0.7335
Epoch 82/100
25724/25724 [==============================] - 1s 48us/sample - loss: 0.5480 - accuracy: 0.7345
Epoch 83/100
25724/25724 [==============================] - 1s 48us/sample - loss: 0.5480 - accuracy: 0.7338
Epoch 84/100
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5478 - accuracy: 0.7341
Epoch 85/100
25724/25724 [======

In [82]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/1 - 0s - loss: 0.5473 - accuracy: 0.7258
Loss: 0.5515596411666092, Accuracy: 0.7258309125900269


In [83]:
# Define model
nn.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 8)                 368       
_________________________________________________________________
dense_29 (Dense)             (None, 5)                 45        
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 6         
Total params: 419
Trainable params: 419
Non-trainable params: 0
_________________________________________________________________
